In [31]:
from iuvs import io

In [32]:
hkfiles = io.get_hk_filenames()

In [33]:
hkfiles[0]

'/maven_iuvs/production/products/housekeeping/level1a/mvn_iuv_analog_l0_20140610_v001.fits.gz'

In [34]:
hk = io.HKReader(hkfiles[0])

In [35]:
sec = hk.AnalogConv['SC_CLK_COARSE']
subsec = hk.AnalogConv['SC_CLK_FINE']

In [36]:
import SpiceyPy as spice

In [37]:
from pathlib import Path
spicedir = Path('/home/klay6683/IUVS-ITF-SW/anc/spice/')
lskfile = spicedir / 'generic_kernels/lsk/naif0011.tls'

True